In [106]:
import numpy as np
import pandas as pd
import warnings
import copy
import random
from scipy.special import softmax
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")

### Read In Data

In [2]:
training_set = pd.read_csv("./fashion-mnist_train.csv")
training_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 359.3 MB


In [3]:
testing_set = pd.read_csv("./fashion-mnist_test.csv")
testing_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 785 entries, label to pixel784
dtypes: int64(785)
memory usage: 59.9 MB


In [4]:
training_set.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
testing_set.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Split into class labels and data points

In [8]:
x_train = training_set.iloc[:, 1:]
y_train = training_set.iloc[:, 0]

In [10]:
x_test = testing_set.iloc[:, 1:]
y_test = testing_set.iloc[:, 0]

### Shuffle Data

In [13]:
np.random.seed(42)

In [14]:
# convert to numpy objects
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

In [17]:
# shuffle training data
arr = np.arange(len(x_train))
np.random.shuffle(arr)
x_train = x_train[arr]
y_train = y_train[arr]

In [19]:
# shuffle testing data
arr = np.arange(len(x_test))
np.random.shuffle(arr)
x_test = x_test[arr]
y_test = y_test[arr]

### Sample Data

In [20]:
# take the first 10K points as the training data
x_train = x_train[:10000].copy()
y_train = y_train[:10000].copy()

In [21]:
# take the first 5K points as the test data
x_test = x_test[:5000].copy()
x_test = x_test[:5000].copy()

# Multiclass Logistic Regression

In [134]:
def LogReg(x_train, y_train, eta, epsilon):
    # map x data to d+1 dimensions
    x = []
    for i in range(len(x_train)):
        x.append(np.insert(x_train[i], 0, 1))
    x = np.array(x)
    
    # create a one hot encoding of the target labels
    y = np.zeros( (y_train.size, y_train.max() + 1) )
    y[np.arange(y_train.size), y_train] = 1
    
    t = 0
    
    # create initial weights matrix
    # K rows and d+1 cols
    w = np.zeros((10, 785) )
    print(y.shape)
    while(True):
        # make a copy of w
        w_old = copy.deepcopy(w)
    
    
        # iterate through dataset in random order
        r = list(range(len(x)))
        random.shuffle(r)
        for i in r:
            
            # get vector of dot products
            prods = np.dot(w, x[i])
            
            # get the softmax for the K dot products
            sm = softmax(prods)
            
            # claculate gradient and update wieghts
            for j in range(len(w)-1):
                grad = (y[i][j] - sm[j]) * x[i]
                new_w = w[j] + eta * grad
                w[j] = new_w
            
        
        # check the break condition
        acc = 0
        for i in range(len(w)-1):
            n = np.linalg.norm(w[j] - w_old[j])
            acc += n
        
        print(acc)
        if acc <= epsilon:
            break
        else: 
            t += 1
    
    return w

In [133]:
weights = LogReg(x_train, y_train, 1e-6, 1e-1)

(10000, 10)
0.3293100692818935
0.16370324701475236
0.1383637351396372
0.13484343598461962
0.1217901600418725
0.10603931707918261
0.10021981013640337
0.12005346583844506
0.1058525531885091
0.1011531315951874
0.10087589808783093
0.09617813194467806
[[ 1.47705877e-04 -2.69091659e-12  3.44375291e-05 ... -1.76120027e-03
  -1.45115626e-03 -3.46533110e-04]
 [-4.57498045e-05 -1.01367336e-15 -2.50291559e-05 ... -1.42275878e-03
  -8.68610080e-04 -2.40905510e-04]
 [-6.76188565e-05 -2.01585508e-06  2.53378681e-05 ...  1.12113789e-03
   9.93859547e-04  6.77935815e-04]
 ...
 [-8.50586533e-05 -2.34790940e-19 -1.65284758e-08 ...  4.20968282e-04
   5.49097294e-04 -4.66741050e-05]
 [-2.31793947e-04  3.38589462e-06  5.40374525e-06 ... -2.93041860e-03
  -1.13748613e-03  2.95982103e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
